In [1]:
import os.path
import numpy as np
import pandas as pd

from pathlib import Path
from edisgo.io.simBEV_import import get_ags, get_ags_data

C:\Users\Kilian.Helfenbein\.conda\envs\eDisGo_env\lib\site-packages\sqlalchemy\ext\declarative\base.py:436: SAWarning: Ignoring declarative-like tuple value of attribute 'segment': possibly a copy-and-paste error with a comma accidentally placed at the end of the line?
  "accidentally placed at the end of the line?" % k
C:\Users\Kilian.Helfenbein\.conda\envs\eDisGo_env\lib\site-packages\sqlalchemy\ext\declarative\base.py:436: SAWarning: Ignoring declarative-like tuple value of attribute 'cable': possibly a copy-and-paste error with a comma accidentally placed at the end of the line?
  "accidentally placed at the end of the line?" % k


In [2]:
data_dir = Path(r"\\192.168.10.221\Daten_flexibel_02\simbev_results")

sub_dir = "standing_times_looped"

scenarios = [
    "Electrification_2050_simbev_run",
    "Electrification_2050_sensitivity_low_work_simbev_run",
    "Mobility_Transition_2050_simbev_run",
    "Szenarette_Kleinwagen_2050_simbev_run",
    "Reference_2050_simbev_run",
    "NEP_C_2035_simbev_run",
]

data_dirs = [
    Path(os.path.join(data_dir, scenario, sub_dir)) for scenario in scenarios
]

In [3]:
df_results = pd.DataFrame(
    columns=[
        "scenario", "ags", "use_case", "charging_processes_count", "mean_standing_time", "mean_charging_time",
        "total_demand", "flex_demand"
    ]
)

for data_dir in data_dirs:
    _, ags_dirs = get_ags(data_dir)
    
    for count, ags_dir in enumerate(ags_dirs):
        dfs = get_ags_data(ags_dir)
        
        for count_dfs, df in enumerate(dfs):
            if count_dfs == 0:
                use_case = "home"
            elif count_dfs == 1:
                use_case = "work"
            elif count_dfs == 2:
                use_case = "public"
            else:
                use_case = "hpc"
                
            if df.empty:
                df_results.loc[len(df_results)] = [
                    ags_dir.parts[-3][:-11],
                    ags_dir.parts[-1],
                    use_case,
                    len(df),
                    np.nan,
                    np.nan,
                    np.nan,
                    np.nan,
                ]
            else:                
                if not count_dfs == 3:
                    df["standing_time"] = df.charge_end - df.charge_start + 1
                    df["charging_time"] = df.chargingdemand.divide(
                        df.netto_charging_capacity.divide(4)
                    ).round(1).apply(np.ceil).astype(int)

                    if count_dfs == 2:
                        demand_flex = 0
                        demand_total = df.chargingdemand.sum()
                    else:
                        demand_flex = df[df.standing_time > df.charging_time].chargingdemand.sum()
                        demand_total = df.chargingdemand.sum()

                    df_results.loc[len(df_results)] = [
                        ags_dir.parts[-3][:-11],
                        ags_dir.parts[-1],
                        use_case,
                        len(df),
                        df.standing_time.mean(),
                        df.charging_time.mean(),
                        demand_total,
                        demand_flex,
                    ]
                else:
                    df_results.loc[len(df_results)] = [
                        ags_dir.parts[-3][:-11],
                        ags_dir.parts[-1],
                        use_case,
                        len(df),
                        1,
                        1,
                        df.chargingdemand.sum(),
                        0,
                    ]

                
        print("AGS {} in scenario {} is done.".format(ags_dir.parts[-1], ags_dir.parts[-3][:-11]))
                    
        del dfs, df

    df_results.to_csv("simbev_analysis_standing_{}.csv".format(ags_dir.parts[-3][:-11]))

df_results = df_results.set_index(["scenario", "ags"])
df_results.to_csv("simbev_analysis.csv")

AGS 03351002 in scenario Electrification_2050 is done.
AGS 03351003 in scenario Electrification_2050 is done.
AGS 03351008 in scenario Electrification_2050 is done.
AGS 03351015 in scenario Electrification_2050 is done.
AGS 03351016 in scenario Electrification_2050 is done.
AGS 03351017 in scenario Electrification_2050 is done.
AGS 08115029 in scenario Electrification_2050 is done.
AGS 08115041 in scenario Electrification_2050 is done.
AGS 08115054 in scenario Electrification_2050 is done.
AGS 08325011 in scenario Electrification_2050 is done.
AGS 08325049 in scenario Electrification_2050 is done.
AGS 08325060 in scenario Electrification_2050 is done.
AGS 08325069 in scenario Electrification_2050 is done.
AGS 09184119 in scenario Electrification_2050 is done.
AGS 09471133 in scenario Electrification_2050 is done.
AGS 09674130 in scenario Electrification_2050 is done.
AGS 09674184 in scenario Electrification_2050 is done.
AGS 09674190 in scenario Electrification_2050 is done.
AGS 096742

AGS 13076089 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076096 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076109 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076120 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076135 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076151 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076164 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076165 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 13076166 in scenario Szenarette_Kleinwagen_2050 is done.
AGS 03351002 in scenario Reference_2050 is done.
AGS 03351003 in scenario Reference_2050 is done.
AGS 03351008 in scenario Reference_2050 is done.
AGS 03351015 in scenario Reference_2050 is done.
AGS 03351016 in scenario Reference_2050 is done.
AGS 03351017 in scenario Reference_2050 is done.
AGS 08115029 in scenario Reference_2050 is done.
AGS 08115041 in scenario Reference_2050 is done.
AGS 08115054 in scenario Reference_2050 is done.
AGS 083250

In [30]:
df_analysis = df_results.copy()

df_flex_share = pd.DataFrame(
    columns=[
        "scenario",
        "ags",
        "flex_share"
    ]
)

for scenario in df_analysis.index.levels[0].tolist():
    df = df_analysis.copy().loc[pd.IndexSlice[[scenario], :], :]
    
    for ags in df.index.levels[1].tolist():
        df_ags = df.loc[pd.IndexSlice[:, [ags]], :]
        
        df_flex_share.loc[len(df_flex_share)] = [
            scenario,
            ags,
            df_ags.flex_demand.sum() / df_ags.total_demand.sum(),
        ]


In [31]:
df_flex_share = df_flex_share.set_index(["scenario", "ags"])

In [32]:
df_flex_share["unflex_share"] = 1 - df_flex_share["flex_share"]

In [37]:
df_flex_share[df_flex_share.index.get_level_values("scenario").isin(["Electrification_2050_sensitivity_low_work"])].describe()

,flex_share,unflex_share
count,35.000000,35.000000
mean,0.660588,0.339412
std,0.022848,0.022848
min,0.613104,0.275662
25%,0.650420,0.327568
50%,0.657659,0.342341
75%,0.672432,0.349580
max,0.724338,0.386896


In [36]:
print(len(df_flex_share))
print(len(df_flex_share[~df_flex_share.index.get_level_values("scenario").isin(["Electrification_2050_sensitivity_low_work"])]))
print(len(df_flex_share[df_flex_share.index.get_level_values("scenario").isin(["Electrification_2050_sensitivity_low_work"])]))

210
175
35
